## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [7]:
import pandas as pd
import json
import requests
from tqdm import tqdm
import concurrent.futures

# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def fetch_option_data(option_name):
    """Fetch the option data for a given option name."""
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin, filtered for options with settlement in months
    """
    # get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # filter for options that have settlement period in 'day'
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == 'month']

    # initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = 'month'
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # finalize data frame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df

# download data -- BTC and ETH Options
data = get_option_data('BTC')


100%|█████████▉| 608/610 [01:17<00:00, 18.33it/s]C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_17668\3326681201.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 610/610 [01:17<00:00,  7.83it/s]


#### Extract Expiration date and strike price

In [8]:
import re
# Define a function to extract expiration date and strike price
def extract_details(instrument_name):
    match = re.match(r"BTC-(\d+[A-Z]{3}\d+)-(\d+)-[CP]", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        return expiration_date, strike_price
    return None, None
data['Expiration Date'], data['Strike Price'] = zip(*data['instrument_name'].apply(extract_details))

data.to_csv('data/btc_data.csv', index=False)
#eth_data.to_csv('data/eth_data.csv', index=False)
